# Class 7: Discrete Derivatives as Convolutional Filters

## Preliminaries

Run the cell below to download the course library and class resources.

In [ ]:
import gdown

gdown.download(id='1SzvuBYIZ407c9eOChXD48NG94v7azJby')
gdown.download(id='1K8nNMjlhWPTgL4OyozYCLaduGmvx1WO-')
gdown.download(id='1_GLvNhIeYdN3kjyfO7jPSw6gocI9547c')

!unzip -o '06.zip'
!rm '06.zip'

!unzip -o '07.zip'
!rm '07.zip'

Run the cell below to import the class modules.

If you get import warnings, try using **`Ctrl+M .`** to restart the kernel. *(notice there is a dot there)*

In [ ]:
import numpy as np
import cv2 as cv

from scipy import signal
from sdx import *

## Choosing, loading, and displaying an image

Like in Class 6, the options of source are: `smash`, `atletica`, `consulting`, `insper`, `informatica`, and `harvard`.



In [ ]:
SOURCE = 'insper'

We will go back to the noise severities at the end of this handout.

In [ ]:
input = cv_grayread(f'{SOURCE}.png', asfloat=True)

cv_imshow(input)

## Convenience functions

Thanks to the `signal` module of the SciPy library, we can have an easy and fast implementation of the 2D convolution we implemented in Class 6. The `mode='valid'` parameter is to specifically consider the approach of ignoring the borders.

In [ ]:
def convolve(input, kernel):
    return signal.convolve2d(input, kernel, mode='valid')

For convenience, let's also define a normalization function, since the outputs of this notebook will often be outside the `[0, 255]` bounds.

In [ ]:
def normalize(output):
    min = output.min()
    max = output.max()
    return 255 * (output - min) / (max - min)

## Sobel filters

The Sobel filters calculate the image gradient as the composition of two convolutions.

The first convolution approximates the discrete derivative for the horizontal axis.

In [ ]:
Gx = np.array([
    [-1,  0,  1],
    [-2,  0,  2],
    [-1,  0,  1],
])

In [ ]:
gx = convolve(input, Gx)

cv_imshow(normalize(gx))
cv_imshow(normalize(np.abs(gx)))

The second convolution approximates the discrete derivative for the vertical axis.

In [ ]:
Gy = np.array([
    [-1, -2, -1],
    [ 0,  0,  0],
    [ 1,  2,  1],
])

In [ ]:
gy = convolve(input, Gy)

cv_imshow(normalize(gy))
cv_imshow(normalize(np.abs(gy)))

Considering the two results as a vector, the intensity can be taken as the module of such vector.

In [ ]:
g = np.sqrt(gx ** 2 + gy ** 2)

cv_imshow(normalize(g))

## Laplacian filter

The second derivative can be approximated by a convolution with the kernel below.

Notice that the result is much more strict than what we got with the Sobel filters.

In [ ]:
L = np.array([
    [ 0,  1,  0],
    [ 1, -4,  1],
    [ 0,  1,  0],
])

In [ ]:
l = convolve(input, L)

cv_imshow(normalize(np.abs(l)))

## Bringing back the issue of noise

When we bring back the issue of noise, an obvious problem is noticed: noises are somewhat sudden variations. Therefore, they give strong responses to an edge detector.

In [ ]:
SEVERITY = 8

In [ ]:
noise_input = cv_grayread(f'{SOURCE}-{SEVERITY}.png', asfloat=True)

cv_imshow(noise_input)

In [ ]:
nl = convolve(noise_input, L)

cv_imshow(normalize(np.abs(nl)))

This can be mitigated, however, by smoothing the image before detecting the edges. The `signal` module also provides a fast implementation of the Gaussian kernel of Class 6.

In [ ]:
def gaussian_kernel(n, sigma, normal=True):
    gaussian1d = signal.windows.gaussian(n, sigma)
    gaussian2d = np.outer(gaussian1d, gaussian1d) / (2 * np.pi * sigma ** 2)
    if normal:
        gaussian2d /= gaussian2d.sum()
    return gaussian2d

In [ ]:
s = convolve(noise_input, gaussian_kernel(3, 1))

sl = convolve(s, L)

cv_imshow(normalize(np.abs(sl)))

## Challenge

Try to obtain the result above with **a single convolution**.

In other words, try to build a "super kernel" that implements both operations at once.

In [ ]:
LoG_k = np.array([
    [ 0,  0, -1, 0,  0],
    [ 0, -1, -2, -1, 0],
    [-1, -2, 16, -2, -1],
    [ 0, -1, -2, -1, 0],
    [ 0,  0, -1, 0,  0],
])

#https://homepages.inf.ed.ac.uk/rbf/HIPR2/log.htm
#https://dsp.stackexchange.com/questions/56034/python-calculating-laplacian-of-gaussian-kernel-matrix

In [ ]:
noise_output = convolve(noise_input.copy(), LoG_k)

cv_imshow(normalize(np.abs(noise_output)))

You can click on the toc.png tab to the left to browse by section.